In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# parameter keys
PARAM_YEAR = 'ww_x_PERIODE_ACAD'
PARAM_STATUS = 'ww_x_PERIODE_PEDAGO'
PARAM_SEMESTER = 'ww_x_HIVERETE'
PARAM_GPS = 'ww_x_GPS'

In [3]:
FILTER_BASE_URL = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847'
GPS_BASE_URL = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_b_list=1&ww_x_UNITE_ACAD=249847'

In [4]:
# semester types
SEMESTER_AUTUMN = 'Autumn'
SEMESTER_SPRING = 'Spring'

In [5]:
# status types
STATUS_B1 = 'B1'
STATUS_B6 = 'B6'

In [6]:
def param_in_url(param_name, param_value):
    return '&' + param_name + '=' + param_value

In [7]:
r = requests.get(FILTER_BASE_URL)

In [8]:
# TODO(weiyu): extract statuses instead of using hard code
statuses = {}
statuses[STATUS_B1] = "249108"
statuses[STATUS_B6] = "942175"

In [9]:
# TODO(weiyu): extract years
years = {}
# 2015 - 2016
years[2015] = "213638028"

In [10]:
# TODO(weiyu):
# extract semesters
semesters = {}
semesters[SEMESTER_AUTUMN] = "2936286"
semesters[SEMESTER_SPRING] = "2936295"

In [11]:
def is_valid_gps(gps):
    return gps != "-1"

def extract_gps(content):
    soup = BeautifulSoup(content, 'html.parser')
    elements = soup.find_all('a', class_='ww_x_GPS')
    for element in elements:
        raw_info = element.attrs['onclick']
        gps = raw_info.split("'")[1].split('=')[1]
        if is_valid_gps(gps):
            return gps

In [12]:
# TODO: query records by gps & store data to the global dict
def handle_gps(gps, year, status, semester):
    if gps == None:
        return
    url = GPS_BASE_URL + param_in_url(PARAM_GPS, gps)
    r = requests.get(url)
    # print(r.content)  

In [13]:
for year, year_val in years.items():
    for status, status_val in statuses.items():        
        for semester, sms_val in semesters.items():
            # construct urls for query
            url = FILTER_BASE_URL
            url += param_in_url(PARAM_YEAR, year_val)
            url += param_in_url(PARAM_STATUS, status_val)
            url += param_in_url(PARAM_SEMESTER, sms_val)

            # get gps list
            r = requests.get(url)
            gps = extract_gps(r.content)
            print(year, status, semester, gps)
            
            # query by gps & extract data to the global dict
            handle_gps(gps, year, status, semester)

2015 B6 Autumn None
2015 B6 Spring 1650772010
2015 B1 Autumn 1897032870
2015 B1 Spring None
